In [1]:
pip install tqdm

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# import numpy as np

# haha = np.random.random(size=2400*2400)
# haha = haha.reshape([2400,2400])
# np.linalg.pinv(haha)

#0. Initial Setting

In [4]:
import numpy as np
from scipy.special import logsumexp
from scipy.special import softmax
import warnings
warnings.filterwarnings("ignore")

theta1 = np.array([1.5,1,-1.9,-1.8,1]) #RS, FC1,FC2, EC

trans_true = [0.9,0.1]
obser_true = [0.2,0.3]
theta23 = trans_true + obser_true

agent = 2
za_dim =(agent+1)*(2**agent)
za_old = np.arange(za_dim,dtype = int)

TimePeriod= 150
SamplePath = 3000
SubSample = 30

# z_old = np.arange(agent+1,dtype=int) #observation
# a1_old = np.arange(2,dtype = int)
# a2_old = np.arange(2,dtype = int)

num_discrete = 50
x_old = np.linspace(0,1,num_discrete)

za_old = np.kron(np.ones(2,dtype=int),np.kron(np.ones(num_discrete,dtype = int),za_old))
x_old = np.kron(np.ones(2,dtype = int),np.kron(x_old,np.ones(za_dim,dtype=int))) #z,a2_old,a1_old,x,a_new
a_new = np.kron(np.arange(2,dtype=int),np.ones(num_discrete*za_dim,dtype=int))

In [5]:
x_old[num_discrete*12:num_discrete*24]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.02040816, 0.02040816, 0.02040816,
       0.02040816, 0.02040816, 0.02040816, 0.02040816, 0.02040816,
       0.02040816, 0.02040816, 0.02040816, 0.02040816, 0.04081633,
       0.04081633, 0.04081633, 0.04081633, 0.04081633, 0.04081633,
       0.04081633, 0.04081633, 0.04081633, 0.04081633, 0.04081633,
       0.04081633, 0.06122449, 0.06122449, 0.06122449, 0.06122449,
       0.06122449, 0.06122449, 0.06122449, 0.06122449, 0.06122449,
       0.06122449, 0.06122449, 0.06122449, 0.08163265, 0.08163265,
       0.08163265, 0.08163265, 0.08163265, 0.08163265, 0.08163265,
       0.08163265, 0.08163265, 0.08163265, 0.08163265, 0.08163265,
       0.10204082, 0.10204082, 0.10204082, 0.10204082, 0.10204082,
       0.10204082, 0.10204082, 0.10204082, 0.10204082, 0.10204082,
       0.10204082, 0.10204082, 0.12244898, 0.12244898, 0.12244

In [6]:
#print(za_old//(agent+1))
print((za_old//(agent+1))//2) #a1
print((za_old//(agent+1))%2)  #a2
print(za_old%(agent+1))       #z

# np.hstack([np.hstack([a1_old[i] *2 + a2_old for i in range(2)])[j] *3+z_old for j in range(4)])
print((1*2+1)*3+2) #+ x*za_dim + a_new * za_dim*x_dim

[0 0 0 ... 1 1 1]
[0 0 0 ... 1 1 1]
[0 1 2 ... 0 1 2]
11


#1. Generate Dynamic Part and Policy Part

In [7]:
def Dynamic(theta23):

  trans_val = theta23[0:2]
  obser_val = theta23[2:4]
  trans = np.zeros([2,2]) #s_t,s_{t+1}
  trans[:,0] = trans_val
  trans[:,1] = 1- trans[:,0]
  
  observe = np.zeros([2,3,3]) #s_{t+1},z_t,z_{t+1}
  for s in range(2):
    observe[s,0,1] = observe[s,1,0] = observe[s,1,2]= observe[s,2,1] = obser_val[s]
    observe[s,0,0] = 1 - observe[s,0,1] -observe[s,0,2]
    observe[s,1,1] = 1 - observe[s,1,0] -observe[s,1,2]
    observe[s,2,2] = 1 - observe[s,2,0] - observe[s,2,1]

  return trans,observe
#print(Dynamic(theta23))

def Reward(theta1,ag_i): # 3*2*2*100*2,2
  rw = np.zeros([(agent+1)*(2**agent)*num_discrete*2,2])#z*a_{t-1}*x_t*a_{t,i},a_{t,-i}
  #for s in range(2):
  if ag_i==1:
    rw[:,0] = ((za_old%(agent+1)) * theta1[0]/((2+0)**2) - theta1[2] - theta1[4] * (1-((za_old//(agent+1))//2)))*x_old
    rw[:,0] += ((za_old%(agent+1)) * theta1[1]/((2+0)**2) - theta1[2] - theta1[4] * (1-((za_old//(agent+1))//2)))*(1-x_old)

    rw[:,1] = ((za_old%(agent+1)) * theta1[0]/((2+1)**2) - theta1[2] - theta1[4] * (1-((za_old//(agent+1))//2)))*x_old
    rw[:,1] += ((za_old%(agent+1)) * theta1[1]/((2+1)**2) - theta1[2] - theta1[4] * (1-((za_old//(agent+1))//2)))*(1-x_old)

  elif ag_i==2:
    rw[:,0] = ((za_old%(agent+1)) * theta1[0]/((2+0)**2) - theta1[3] - theta1[4] * (1-((za_old//(agent+1))%2)))*x_old
    rw[:,0] += ((za_old%(agent+1)) * theta1[1]/((2+0)**2) - theta1[3] - theta1[4] * (1-((za_old//(agent+1))%2)))*(1-x_old)

    rw[:,1] = ((za_old%(agent+1)) * theta1[0]/((2+1)**2) - theta1[3] - theta1[4] * (1-((za_old//(agent+1))%2)))*x_old
    rw[:,1] += ((za_old%(agent+1)) * theta1[1]/((2+1)**2) - theta1[3] - theta1[4] * (1-((za_old//(agent+1))%2)))*(1-x_old)

  rw[:,0] = a_new * rw[:,0]
  rw[:,1] = a_new * rw[:,1]

  return rw
#print(Reward(theta1,2)[1200:2400])


def SigmaLambda(za_old,x_old,trans,observe,num_discrete=num_discrete,T = None): #(xt,z_t,a_old,a_new_i),z_{t+1}
  if T==None: #generate function for Q function iteration
      x_new = np.zeros([x_old.shape[0],3])
      sigma = np.zeros([x_old.shape[0],3])
      x_old = np.vstack([x_old,1-x_old])
      for z_new in range(3):
        x_new_tep = trans[:,0].reshape([-1,1])
        x_new_tep = x_new_tep.dot(observe[0,za_old%(agent+1),z_new].reshape([1,-1]))
        x_new_tep = np.sum(x_old * x_new_tep,axis=0)
        #print(x_new_tep.shape)
        sigma[:,z_new] = x_new_tep + np.sum(x_old*(np.reshape(trans[:,1],[-1,1]).dot(observe[1,za_old%(agent+1),z_new].reshape([1,-1]))),
                                            axis=0)
        sigma_0 = np.where(sigma[:,z_new]!=0)
        x_new[:,z_new][sigma_0] = x_new_tep[sigma_0] /sigma[:,z_new][sigma_0] 

      belief_f = np.floor(x_new*(num_discrete-1))/(num_discrete-1)
      belief_c = np.ceil(x_new*(num_discrete-1))/(num_discrete-1)
      iterpolate = np.zeros(belief_f.shape)
      iterpolate[np.where((belief_f-belief_c)!=0)] = (x_new-belief_c)[np.where((belief_f-belief_c)!=0)]/(belief_f-belief_c)[np.where((belief_f-belief_c)!=0)]

  else:      #generate whole belief for recover process
      x_new = np.zeros([T,x_old.shape[0]]) # T, N
      sigma = np.zeros([T-1,x_old.shape[0]]) # T-1 ,N
      x_new[0,:] = x_old

      for t in range(1,T):

        za = za_old[t-1,:]
        z = za_old[t,:]
        
        x_new_tep = (x_new[t-1,:] *trans[0,0] + (1-x_new[t-1,:])*trans[1,0] )* observe[0,za,z]
        sigma[t-1,:] = x_new_tep + (x_new[t-1,:] *trans[0,1] + (1-x_new[t-1,:])*trans[1,1] )* observe[1,za,z]
        x_new[t,:] = x_new_tep/sigma[t-1,:]

      belief_f = np.floor(x_new*(num_discrete-1))/(num_discrete-1)
      belief_c = np.ceil(x_new*(num_discrete-1))/(num_discrete-1)
      iterpolate = np.zeros(belief_f.shape)
      iterpolate[np.where((belief_f-belief_c)!=0)] = (x_new-belief_c)[np.where((belief_f-belief_c)!=0)]/(belief_f-belief_c)[np.where((belief_f-belief_c)!=0)]      

  return sigma,x_new,[iterpolate,np.int_(belief_f*(num_discrete-1)),np.int_(belief_c*(num_discrete-1))]
# trans,observe = Dynamic(theta23)  #s_t,s_{t+1} #s_{t+1},z_t,z_{t+1}
# print(SigmaLambda(za_old,x_old,trans,observe)[0].shape)

# trans,observe = Dynamic(theta23)  #s_t,s_{t+1} #s_{t+1},z_t,z_{t+1}
# sigma,x_new,xiterpolate = SigmaLambda(za_old,x_old,trans,observe)

In [13]:
import time
import scipy
def Pifunc(theta1,sigma,xiterpolate, num_discrete=num_discrete,beta = 0.95,
                   gamma = 0.5772,thread_inner = 1e-5,thread_outer = 1e-5): #xt*za*a1 or xt*za*a2
  rw1,rw2 = Reward(theta1,1), Reward(theta1,2) #z*a_{t-1}*xt*a_{t,i},a_{t,-i}
  iterpolate,belief_f,belief_c = xiterpolate[0],xiterpolate[1],xiterpolate[2]#z*a_{t-1}*xt*a_{t,i},a_{t,-i}
  x_old = np.linspace(0,1,num=num_discrete)#.reshape([-1,1])
  x_old = np.kron(np.ones(2,dtype = int),np.kron(x_old,np.ones(za_dim,dtype=int))) #z,a2_old,a1_old,x,a_new  
  #x_old = np.vstack([x_old,1-x_old]).T

  pi1_new = 0.5*np.ones(num_discrete*za_dim*2) #xt*za*a1

  pi2_old = pi1_new.copy()
  #count = 0
  #pi1_old = np.zeros([2,2])
  for ol in range(1000):
    pi1_old = pi1_new.copy()
    
    #Q1_new = np.zeros(num_discrete*za_dim*2) #xt,za,a1
    h1_new = np.zeros(num_discrete*za_dim*2) #xt,za,a1
    g1_new = np.zeros(num_discrete*za_dim*2) #xt,za,a1
    
    h2_new = np.zeros(num_discrete*za_dim*2) #xt,za,a2
    g2_new = np.zeros(num_discrete*za_dim*2) #xt,za,a1


    Idmat1 = np.zeros([num_discrete*za_dim*2,num_discrete*za_dim*2])
    xx = np.arange(num_discrete*za_dim*2,dtype=int)
    for z_new in range(3):
      znew_a10= (0 *2+a_new)*3 + z_new
      it0,bf0,bc0 = iterpolate[:,z_new],belief_f[:,z_new],belief_c[:,z_new]
      #print(znew_a10,bf0)
      Idmat1[xx,bf0*za_dim+znew_a10+0*num_discrete*za_dim] +=    it0*np.kron(np.ones(2),pi2_old[0:num_discrete*za_dim])*sigma[:,z_new]
      Idmat1[xx,bc0*za_dim+znew_a10+0*num_discrete*za_dim] +=(1-it0)*np.kron(np.ones(2),pi2_old[0:num_discrete*za_dim])*sigma[:,z_new]
      Idmat1[xx,bf0*za_dim+znew_a10+1*num_discrete*za_dim] +=    it0*np.kron(np.ones(2),pi2_old[num_discrete*za_dim:])*sigma[:,z_new]
      Idmat1[xx,bc0*za_dim+znew_a10+1*num_discrete*za_dim] +=(1-it0)*np.kron(np.ones(2),pi2_old[num_discrete*za_dim:])*sigma[:,z_new]
    pi1_tep1  = np.kron(np.ones(2),pi1_old[0:num_discrete*za_dim])
    Idmat1 = np.dot(pi1_tep1.reshape([-1,1]),np.ones([1,num_discrete*za_dim*2])) * Idmat1
    
    Idmat2 = np.zeros([num_discrete*za_dim*2,num_discrete*za_dim*2])
    for z_new in range(3):
      znew_a11= (1 *2+a_new)*3 + z_new
      it1,bf1,bc1 = iterpolate[:,z_new],belief_f[:,z_new],belief_c[:,z_new]
      Idmat2[xx,bf1*za_dim+znew_a11+0*num_discrete*za_dim] +=    it1*np.kron(np.ones(2),pi2_old[0:num_discrete*za_dim])*sigma[:,z_new]
      Idmat2[xx,bc1*za_dim+znew_a11+0*num_discrete*za_dim] +=(1-it1)*np.kron(np.ones(2),pi2_old[0:num_discrete*za_dim])*sigma[:,z_new]
      Idmat2[xx,bf1*za_dim+znew_a11+1*num_discrete*za_dim] +=    it1*np.kron(np.ones(2),pi2_old[num_discrete*za_dim:])*sigma[:,z_new]
      Idmat2[xx,bc1*za_dim+znew_a11+1*num_discrete*za_dim] +=(1-it1)*np.kron(np.ones(2),pi2_old[num_discrete*za_dim:])*sigma[:,z_new]
    pi1_tep2  = np.kron(np.ones(2),pi1_old[num_discrete*za_dim:])
    Idmat2 = np.dot(pi1_tep2.reshape([-1,1]),np.ones([1,num_discrete*za_dim*2])) * Idmat2
    
    Idmat1 = beta * (Idmat1 + Idmat2)
    
    for il in range(1000):
      h2_old = h2_new.copy() #xt,za,a2
      g2_old = g2_new.copy() #xt,za,a2
      
      h2_new =   pi1_tep1 * rw2[:,0] + pi1_tep2 * rw2[:,1] + Idmat1.dot(h2_old)      
      g2_new =  Idmat1.dot(gamma-np.log(pi2_old)) + Idmat1.dot(g2_old)
      #print(it0)
      
      #print(il,np.max(np.abs(np.nan_to_num((h2_new-h2_old)/h2_old))) <thread_inner ,np.max(np.abs(np.nan_to_num((g2_new-g2_old)/g2_old))))
      if np.max(np.abs(np.nan_to_num((h2_new-h2_old)/h2_old))) <thread_inner and np.max(np.abs(np.nan_to_num((g2_new-g2_old)/g2_old))) <thread_inner: 
        #print('Q2',il,np.max(np.abs(np.nan_to_num((Q2_new-Q2_old)/Q2_old))))
        break

    pi2_new = np.reshape(softmax(np.reshape(h2_new + g2_new,[2,-1]),axis=0),-1)   
    count2 = np.max(np.abs(np.nan_to_num((pi2_old-pi2_new)/pi2_old)))


    pi2_old = pi2_new.copy()
    
    Idmat1 = np.zeros([num_discrete*za_dim*2,num_discrete*za_dim*2])
    xx = np.arange(num_discrete*za_dim*2,dtype=int)
    for z_new in range(3):
      znew_a20= (a_new*2+0)*3 + z_new
      it0,bf0,bc0 = iterpolate[:,z_new],belief_f[:,z_new],belief_c[:,z_new]
      #print(znew_a10,bf0)
      Idmat1[xx,bf0*za_dim+znew_a20+0*num_discrete*za_dim] +=    it0*np.kron(np.ones(2),pi1_old[0:num_discrete*za_dim])*sigma[:,z_new]
      Idmat1[xx,bc0*za_dim+znew_a20+0*num_discrete*za_dim] +=(1-it0)*np.kron(np.ones(2),pi1_old[0:num_discrete*za_dim])*sigma[:,z_new]
      Idmat1[xx,bf0*za_dim+znew_a20+1*num_discrete*za_dim] +=    it0*np.kron(np.ones(2),pi1_old[num_discrete*za_dim:])*sigma[:,z_new]
      Idmat1[xx,bc0*za_dim+znew_a20+1*num_discrete*za_dim] +=(1-it0)*np.kron(np.ones(2),pi1_old[num_discrete*za_dim:])*sigma[:,z_new]
    pi2_tep1  = np.kron(np.ones(2),pi2_old[0:num_discrete*za_dim])
    Idmat1 = np.dot(pi2_tep1.reshape([-1,1]),np.ones([1,num_discrete*za_dim*2])) * Idmat1
    
    Idmat2 = np.zeros([num_discrete*za_dim*2,num_discrete*za_dim*2])
    for z_new in range(3):
      znew_a21= (a_new*2+1)*3 + z_new
      it1,bf1,bc1 = iterpolate[:,z_new],belief_f[:,z_new],belief_c[:,z_new]
      Idmat2[xx,bf1*za_dim+znew_a21+0*num_discrete*za_dim] +=    it1*np.kron(np.ones(2),pi1_old[0:num_discrete*za_dim])*sigma[:,z_new]
      Idmat2[xx,bc1*za_dim+znew_a21+0*num_discrete*za_dim] +=(1-it1)*np.kron(np.ones(2),pi1_old[0:num_discrete*za_dim])*sigma[:,z_new]
      Idmat2[xx,bf1*za_dim+znew_a21+1*num_discrete*za_dim] +=    it1*np.kron(np.ones(2),pi1_old[num_discrete*za_dim:])*sigma[:,z_new]
      Idmat2[xx,bc1*za_dim+znew_a21+1*num_discrete*za_dim] +=(1-it1)*np.kron(np.ones(2),pi1_old[num_discrete*za_dim:])*sigma[:,z_new]
    pi2_tep2  = np.kron(np.ones(2),pi2_old[num_discrete*za_dim:])
    Idmat2 = np.dot(pi2_tep2.reshape([-1,1]),np.ones([1,num_discrete*za_dim*2])) * Idmat2
    
    Idmat1 = beta * (Idmat1 + Idmat2)
    
    for il in range(1000):
      h1_old = h1_new.copy() #xt,za,a2
      g1_old = g1_new.copy() #xt,za,a2


      h1_new =   pi2_tep1 * rw1[:,0] + pi2_tep2 * rw1[:,1] + Idmat1.dot(h1_old)      
      g1_new =  Idmat1.dot(gamma-np.log(pi1_old)) + Idmat1.dot(g1_old)
      
      #print(it0)
      
      #print(il,np.max(np.abs(np.nan_to_num((h1_new-h1_old)/h1_old))) <thread_inner ,np.max(np.abs(np.nan_to_num((g1_new-g1_old)/g1_old))))
      if np.max(np.abs(np.nan_to_num((h1_new-h1_old)/h1_old))) <thread_inner and np.max(np.abs(np.nan_to_num((g1_new-g1_old)/g1_old))) <thread_inner: 
        #print('Q1',il,np.max(np.abs(np.nan_to_num((Q1_new-Q1_old)/Q1_old))))
        break

    pi1_new = np.reshape(softmax(np.reshape(h1_new + g1_new,[2,-1]),axis=0),-1)   
    count1 = np.max(np.abs(np.nan_to_num((pi1_old-pi1_new)/pi1_old)))
    
    print('pi1 pi2',ol,count1,count2)

    if count1 <thread_outer and count2<thread_outer:
      #print(ol,np.max(np.abs((pi1_old-pi1_new)/pi1_new)))
      break
  return pi1_new,pi2_new,[h1_new,g1_new],[h2_new,g2_new]

In [14]:
trans,observe = Dynamic(theta23)  #s_t,s_{t+1} #s_{t+1},z_t,z_{t+1}
sigma,x_new,xiterpolate = SigmaLambda(za_old,x_old,trans,observe)
#pi1,pi2 = Pifunc(theta1,sigma,xiterpolate)

max_item = 1000
thread = 1e-5

pi1_old = 0.5*np.ones(num_discrete*za_dim*2)
pi2_old = 0.5*np.ones(num_discrete*za_dim*2)

for i in range(max_item):
    pi1_new,pi2_new,_,_ = Pifunc(theta1,sigma,xiterpolate)

    error1 = np.max(np.abs((pi1_new[0:1200]-pi1_old[0:1200])/pi1_old[0:1200]))
    error2 = np.max(np.abs((pi2_new[0:1200]-pi2_old[0:1200])/pi2_old[0:1200]))

    print('error: ',error1,error2)

    if error1 <=thread and error2<=thread:
      break
    pi1_old = pi1_new
    pi2_old = pi2_new
    print(pi1_old)
    print(pi2_old)

pi1 pi2 0 0.8816878123242062 0.8871396852913911
pi1 pi2 1 0.319605405931469 0.2943453496000092
pi1 pi2 2 0.003702099131117577 0.021046153534364703
pi1 pi2 3 1.7916908342351788e-05 6.097425086629752e-05
pi1 pi2 4 6.26147358571677e-07 2.3405169676564405e-06
error:  0.9188801522179659 0.909717285763534
[0.14671778 0.13023735 0.11581756 ... 0.94050348 0.95024322 0.958278  ]
[0.16092078 0.14325646 0.12770302 ... 0.93409169 0.94481116 0.95368645]
pi1 pi2 0 0.8816878123242062 0.8871396852913911
pi1 pi2 1 0.319605405931469 0.2943453496000092
pi1 pi2 2 0.003702099131117577 0.021046153534364703
pi1 pi2 3 1.7916908342351788e-05 6.097425086629752e-05
pi1 pi2 4 6.26147358571677e-07 2.3405169676564405e-06
error:  0.0 0.0


#2. Data Generation Process

In [15]:
trans,observe = Dynamic(theta23)  #s_t,s_{t+1} #s_{t+1},z_t,z_{t+1}
#trans-np.diag(np.ones(2))

In [16]:
##################################Global Parameter#################################
# theta1 = np.array([3,1.9,2]) #RS, FC, EC
# theta23 = 0.2
# agent = 2
# za_dim =(agent+1)*(2**agent)
# za_old = np.arange(z_dim,dtype = int)
# TimePeriod= 150
# SamplePath = 300
# num_discrete = 100
# z_old = np.arange(agent+1,dtype=int) #observation
# a1_old = np.arange(2,dtype = int)
# a2_old = np.arange(2,dtype = int)
###################################################################################
from tqdm import tqdm 

def DataGeneration(theta1 = theta1,theta23=theta23,T=TimePeriod,N=SamplePath,N_sub = SubSample):
  # Assume start from good condition and highest market size(2), agents entry market(1) at time 0

  trans,observe = Dynamic(theta23)  #s_t,s_{t+1} #s_{t+1},z_t,z_{t+1}
  sigma,x_new,xiterpolate = SigmaLambda(za_old,x_old,trans,observe)
  pi1,pi2,_,_ = Pifunc(theta1,sigma,xiterpolate)#xt,za, a1 or xt,za, a2


  condition = np.zeros([T,N],dtype= int)
  condition[0,:] = np.random.choice(np.array(2,dtype=int),size=N,p=[1/2,1/2])

  market_size = np.zeros([T,N],dtype = int) #observation state
  market_size[0,:] = np.random.choice(np.array(3,dtype=int),size=N,p=[1/3,1/3,1/3])

  belief = np.zeros([T,N])
  belief[0,:] = np.kron(np.linspace(0,1,num=int(N/N_sub)),np.ones(N_sub))
  sigma_val = np.ones([T-1,N])
  belief_f = np.floor(belief[0,:]*(num_discrete-1))/(num_discrete-1)
  belief_c = np.ceil(belief[0,:]*(num_discrete-1))/(num_discrete-1)
  iterpolate = np.zeros(belief_f.shape)
  iterpolate[np.where((belief_f-belief_c)!=0)] = (belief[0,:]-belief_c)[np.where((belief_f-belief_c)!=0)]/(belief_f-belief_c)[np.where((belief_f-belief_c)!=0)]      
  belief_f,belief_c = np.int_(belief_f*(num_discrete-1)),np.int_(belief_c*(num_discrete-1))

  action1 = np.ones([T,N],dtype = int)   #agent1's action
  action2 = np.ones([T,N],dtype = int)   #agent2's action
  # za = (np.zeros(N,dtype=int) *2 + np.zeros(N,dtype=int))*3+market_size[0,:]
  # action_mat1 = np.random.random(size=N)
  # p1 = iterpolate * pi1[belief_f,za,0] + (1-iterpolate) * pi1[belief_c,za,0]
  # action1[0,:] = np.int_(p1<=action_mat1)
  
  # action_mat2 = np.random.random(size=N)
  # p2 = iterpolate * pi2[belief_f,za,0] + (1-iterpolate) * pi2[belief_c,za,0]
  # action2[0,:] = np.int_(p2<=action_mat2)


  for t in tqdm(range(1,T)):
    #generate new condition #s_{t+1}
    cd_mat = np.random.random(size=N)
    pc = trans[condition[t-1,:],0]
    condition[t,:] = np.int_(pc<=cd_mat)
   
    # generate new market size #z_{t+1}
    ms_mat = np.random.random(size=N)
    pt = observe[condition[t,:],market_size[t-1,:],:]
    market_size[t,:] = np.int_(pt[:,0]<=ms_mat) +np.int_(pt[:,0]+pt[:,1]<=ms_mat)

    # update new belief #x_t
    za = market_size[t-1,:]
    z = market_size[t,:]
    x_new_tep = (belief[t-1,:] *trans[0,0] + (1-belief[t-1,:])*trans[1,0] )* observe[0,za,z]
    #print(x_new_tep)
    sigma_val[t-1,:] = x_new_tep + (belief[t-1,:] *trans[0,1] + (1-belief[t-1,:])*trans[1,1] )* observe[1,za,z]
    belief[t,:] = x_new_tep/sigma_val[t-1,:]

    belief_f = np.floor(belief[t,:]*(num_discrete-1))/(num_discrete-1)
    belief_c = np.ceil(belief[t,:]*(num_discrete-1))/(num_discrete-1)
    iterpolate = np.zeros(belief_f.shape)
    iterpolate[np.where((belief_f-belief_c)!=0)] = (belief[t,:]-belief_c)[np.where((belief_f-belief_c)!=0)]/(belief_f-belief_c)[np.where((belief_f-belief_c)!=0)]      
    belief_f,belief_c = np.int_(belief_f*(num_discrete-1)),np.int_(belief_c*(num_discrete-1))

    # generate new action for both agents #a_{t+1}
    za = (action1[t-1,:] *2 + action2[t-1,:])*3+market_size[t,:]
    action_mat1 = np.random.random(size=N)
    p1 = iterpolate * pi1[belief_f*za_dim+za+0] + (1-iterpolate) * pi1[belief_c*za_dim+za+0]
    action1[t,:] = np.int_(p1<=action_mat1)
    
    action_mat2 = np.random.random(size=N)
    p2 = iterpolate * pi2[belief_f*za_dim+za+0]+ (1-iterpolate) * pi2[belief_c*za_dim+za+0]
    action2[t,:] = np.int_(p2<=action_mat2)

  return market_size, action1,action2 ,belief,sigma_val,condition


market_size, action1,action2 ,belief,sigma_val,condition = DataGeneration()
#haha = SigmaLambda()

pi1 pi2 0 0.8816878123242062 0.8871396852913911
pi1 pi2 1 0.319605405931469 0.2943453496000092
pi1 pi2 2 0.003702099131117577 0.021046153534364703
pi1 pi2 3 1.7916908342351788e-05 6.097425086629752e-05


100%|██████████| 149/149 [00:00<00:00, 1115.70it/s]

pi1 pi2 4 6.26147358571677e-07 2.3405169676564405e-06


In [17]:
np.sum(condition)/np.prod(condition.shape)

0.49941555555555556

#3.1 Recover Dynamic Part

In [18]:
theta23

[0.9, 0.1, 0.2, 0.3]

In [19]:
from scipy.optimize import minimize
ini_x0 = 0.85*np.array(theta23)
print('start from:',ini_x0 )

def func1(theta23,market_size=market_size, action1=action1,action2 = action2,T=TimePeriod ,x0 = belief[0,:]):
  trans,observe = Dynamic(theta23)  #s_t,s_{t+1} #s_{t+1},z_t,z_{t+1}
  sigma_val,_,_ = SigmaLambda(market_size,x0,trans,observe,T=T)

  # sigma_val,_,_ = SigmaLambda(None,market_size,x_old,action1,action2,theta23= theta23,num_discrete=num_discrete,T = T)
  #print(observe[market_size[0:-1,:],market_size[1:T,:]])
  res = -np.sum(np.log(sigma_val))
  #print(theta23,res)
  return res

res1 = minimize(func1,ini_x0, bounds=[[0.00001,0.99999],[0.00001,0.99999],[0.00001,0.49999],[0.00001,0.49999]])
#trans_est = np.vstack([res1.x,1-res1.x]).T.reshape([2,2,2,2])
print(res1)

start from: [0.765 0.085 0.17  0.255]
      fun: 322043.6849301086
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.21536835, -0.12805685,  0.64028427,  0.25611371])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 135
      nit: 22
   status: 0
  success: True
        x: array([0.89105871, 0.12071396, 0.20092711, 0.30372631])


#3.2 Recover Reward Part

In [20]:
theta1.reshape(-1)

array([ 1.5,  1. , -1.9, -1.8,  1. ])

In [21]:
ini_x0 = 0.85*theta1.reshape(-1)
print('start from:',ini_x0 )

trans,observe = Dynamic(theta23)  #s_t,s_{t+1} #s_{t+1},z_t,z_{t+1}
_,belief_val,_ = SigmaLambda(market_size,belief[0,:],trans,observe,T=TimePeriod)
sigma,x_new,xiterpolate = SigmaLambda(za_old,x_old,trans,observe)

def func2(theta1,sigma = sigma,xiterpolate=xiterpolate,belief_val=belief_val,market_size=market_size,action1 = action1,action2 = action2,T=TimePeriod,x0 = belief[0,:]):
  #theta1 = theta1.reshape([2,2,2])  
 
  pi1,pi2,_,_ = Pifunc(theta1,sigma,xiterpolate)
  
  za = (action1[0:-1,:]*2 + action2[0:-1,:])*3 + market_size[1:T,:]


  belief_f = np.floor(belief_val[1:T,:]*(num_discrete-1))/(num_discrete-1)
  belief_c = np.ceil(belief_val[1:T,:]*(num_discrete-1))/(num_discrete-1)
  iterpolate = np.zeros(belief_f.shape)
  iterpolate[np.where((belief_f-belief_c)!=0)] = (belief_val[1:T,:]-belief_c)[np.where((belief_f-belief_c)!=0)]/(belief_f-belief_c)[np.where((belief_f-belief_c)!=0)]      
  belief_f,belief_c = np.int_(belief_f*(num_discrete-1)),np.int_(belief_c*(num_discrete-1))

  p1 = iterpolate*pi1[belief_f*za_dim+za+action1[1:T,:]*num_discrete*za_dim] +\
  (1- iterpolate)*pi1[belief_c*za_dim+za+action1[1:T,:]*num_discrete*za_dim]
  p2 = iterpolate*pi2[belief_f*za_dim+za+action2[1:T,:]*num_discrete*za_dim]+\
  (1- iterpolate)*pi2[belief_c*za_dim+za+action2[1:T,:]*num_discrete*za_dim]
  res = - np.sum(np.log(p1)) -np.sum(np.log(p2))
  print(theta1,res)
  return res
# ineq_cons = {'type': 'ineq',
#              'fun' : lambda x: np.array([x[0] - x[1]])}
res2 = minimize(func2,ini_x0,bounds = [(1e-6,100),(1e-6,100),(-100,-1e-6),(-100,-1e-6),(1e-6,100)])#,constraints=ineq_cons)#,(1e-6,100)])#method='Nelder-Mead')
print(res2)

start from: [ 1.275  0.85  -1.615 -1.53   0.85 ]
pi1 pi2 0 0.8281428065899044 0.8327627994619395
pi1 pi2 1 0.2544087649891898 0.22763855830742016
pi1 pi2 2 0.0016881783972402347 0.01649006609030911
pi1 pi2 3 1.5875450016092478e-05 8.593858021538347e-05
pi1 pi2 4 7.594449413921235e-07 3.3884515740393685e-06
[ 1.275  0.85  -1.615 -1.53   0.85 ] 204192.38968324076
pi1 pi2 0 0.8281428069866359 0.8327628000153593
pi1 pi2 1 0.2544087651414841 0.22763855830859997
pi1 pi2 2 0.001688178398322175 0.01649006619978925
pi1 pi2 3 1.5875450286269368e-05 8.59385807696684e-05
pi1 pi2 4 7.594450621778353e-07 3.3884515527372328e-06
[ 1.27500001  0.85       -1.615      -1.53        0.85      ] 204192.38966743648
pi1 pi2 0 0.8281428065899035 0.8327627994619395
pi1 pi2 1 0.25440876500198917 0.2276385583246414
pi1 pi2 2 0.0016881783982581383 0.016490066086864024
pi1 pi2 3 1.5875449980534035e-05 8.593857971793654e-05
pi1 pi2 4 7.594450691222349e-07 3.388451893815366e-06
[ 1.275       0.85000001 -1.615      -1

In [ ]:
ini_x0 = 0.85*theta1.reshape(-1)
print('start from:',ini_x0 )

trans,observe = Dynamic(res1.x)  #s_t,s_{t+1} #s_{t+1},z_t,z_{t+1}
_,belief_val,_ = SigmaLambda(market_size,belief[0,:],trans,observe,T=TimePeriod)
sigma,x_new,xiterpolate = SigmaLambda(za_old,x_old,trans,observe)

def func2(theta1,sigma = sigma,xiterpolate=xiterpolate,belief_val=belief_val,market_size=market_size,action1 = action1,action2 = action2,T=TimePeriod,x0 = belief[0,:]):
  #theta1 = theta1.reshape([2,2,2])  
 
  _,_,pi1,pi2 = ValueIteration(theta1,sigma,xiterpolate)
  
  za = (action1[0:-1,:]*2 + action2[0:-1,:])*3 + market_size[1:T,:]


  belief_f = np.floor(belief_val[1:T,:]*(num_discrete-1))/(num_discrete-1)
  belief_c = np.ceil(belief_val[1:T,:]*(num_discrete-1))/(num_discrete-1)
  iterpolate = np.zeros(belief_f.shape)
  iterpolate[np.where((belief_f-belief_c)!=0)] = (belief_val[1:T,:]-belief_c)[np.where((belief_f-belief_c)!=0)]/(belief_f-belief_c)[np.where((belief_f-belief_c)!=0)]      
  belief_f,belief_c = np.int_(belief_f*(num_discrete-1)),np.int_(belief_c*(num_discrete-1))

  p1 = iterpolate*pi1[belief_f*za_dim+za+action1[1:T,:]*num_discrete*za_dim] +\
  (1- iterpolate)*pi1[belief_c*za_dim+za+action1[1:T,:]*num_discrete*za_dim]
  p2 = iterpolate*pi2[belief_f*za_dim+za+action2[1:T,:]*num_discrete*za_dim]+\
  (1- iterpolate)*pi2[belief_c*za_dim+za+action2[1:T,:]*num_discrete*za_dim]
  res = - np.sum(np.log(p1)) -np.sum(np.log(p2))
  print(theta1,res)
  return res
# ineq_cons = {'type': 'ineq',
#              'fun' : lambda x: np.array([x[0] - x[1]])}
res2 = minimize(func2,ini_x0,bounds = [(1e-6,100),(1e-6,100),(-100,-1e-6),(-100,-1e-6),(1e-6,100)])#,constraints=ineq_cons)#,(1e-6,100)])#method='Nelder-Mead')
print(res2)